<a href="https://colab.research.google.com/github/manlio99/Materia-de-aprendizaje/blob/master/7_FinalProjects/H2O_auto_ML/h2o_automl_completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h2 align="center">Automatic Machine Learning with H2O AutoML</h2>

In [ ]:
!apt-get install openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  gvfs openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin
  libgtk2.0-common libxxf86dga1 openjdk-8-jdk openjdk-8-jdk-headless
  openjdk-8-jre openjdk-8-jre-headless x11-utils
0 upgraded, 15 newly installed, 0 to remove and 21 not upgraded.
Need to get 42.8 MB of archives.
After this 

In [ ]:
!java -version

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode)


This command displays Java version.

Make sure it displays openjdk version 1.8.*

If it displays otherwise, (such as openjdk version 10.*) H2O might display an error like this:

<i>H2OServerError: Server process terminated with error code 1</i>

To fix the issue, try this command to change Java version.

```!update-alternatives --config java```

In [ ]:
sudo update-alternatives --set java /usr/lib/jvm/jdk1.8.0_version/bin/java
#!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
* 0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
  2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: ^C


In [ ]:
!java -version

openjdk version "11.0.8" 2020-07-14
OpenJDK Runtime Environment (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.8+10-post-Ubuntu-0ubuntu118.04.1, mixed mode)


In [ ]:
!pip install pandas==0.19.2

     |████████████████████████████████| 18.9MB 1.2MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.19.2 which is incompatible.
ERROR: seaborn 0.11.0 has requirement pandas>=0.23, but you'll have pandas 0.19.2 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.19.2 which is incompatible.
ERROR: pandas-datareader 0.9.0 has requirement pandas>=0.23, but you'll have pandas 0.19.2 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.19.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 0.19.2 which is incompatible.
ERROR: fbprophet 0.7.1 has requirement pandas>=1.0.4, but you'll have pandas 0.19.2 which is incompatible.
  Found existing installation: pandas 1.1.2
    Uninstalling pandas-1.1.2:
      Successfully uninstalled pandas-1.1.2


In [ ]:
!pip install H2O

     |████████████████████████████████| 129.4MB 97kB/s 


In [ ]:
import h2o
h2o.init()
#memoria limitada a 2gb y 1 hr de entrenamiento

### Task 2: Importing Packages

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files

uploaded = files.upload()
#Subir data


### Task 3: Loading and Exploring the Data

In [ ]:
xls = pd.ExcelFile('/content/bank_term_deposit_marketing_analysis.xlsx')

In [ ]:
xls.sheet_names
#hojas de excel

In [ ]:
#cargamos cada hoja de excel como un dataframe separado
client_info = pd.read_excel(xls, 'CLIENT_INFO')
loan_history = pd.read_excel(xls, 'LOAN_HISTORY')
marketing_history = pd.read_excel(xls, 'MARKETING HISTORY')
subscription_history = pd.read_excel(xls, 'SUBSCRIPTION HISTORY')

In [ ]:
client_info.head()
#verificamos que esten bien cargados

In [ ]:
loan_history.head()

In [ ]:
marketing_history.head()

In [ ]:
subscription_history.head()

In [ ]:
df = pd.merge(client_info, loan_history, on=['ID'])
df = pd.merge(df, marketing_history, on=['ID'])
df = pd.merge(df, subscription_history, on=['ID'])
df.head()
#unimos todos los dataframes en uno solo

In [ ]:
df = df.drop(['ID'], axis=1)
#eliminamos la columna id porque no aporta informacion

### Task 4: Data Prep & Start H2O

In [ ]:
import multiprocessing
multiprocessing.cpu_count()

In [ ]:
import psutil
psutil.virtual_memory()

In [ ]:
#import h2o
from h2o.automl import H2OAutoML

In [ ]:
h2o_df = h2o.H2OFrame(df)

In [ ]:
h2o_df.describe()

In [ ]:
train, test = h2o_df.split_frame(ratios=[.75])

# Identify predictors and response
x = train.columns
y = "TERM_DEPOSIT"
x.remove(y)

### Task 5: Run H2O AutoML

In [ ]:
from h2o.automl import H2OAutoML

In [ ]:
aml = H2OAutoML(max_runtime_secs=600,
                #exclude_algos=['DeepLearning'],
                seed=1,
                #stopping_metric='logloss',
                #sort_metric='logloss',
                balance_classes=False,
                project_name='Completed'
)
%time aml.train(x=x, y=y, training_frame=train)

### Task 6: AutoML Leaderboard and Ensemble Exploration

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

In [ ]:
# Get the "All Models" Stacked Ensemble model
se = aml.leader

# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
metalearner.varimp()

### Task 7: Baselearner Model Exploration

In [ ]:
model = h2o.get_model('XGBoost_grid__1_AutoML_20200608_075205_model_2')

In [ ]:
model.model_performance(test)

In [ ]:
model.varimp_plot(num_of_features=20)

In [ ]:
model.partial_plot(train, cols=["DURATION"], figsize=(5,5));

In [ ]:
# save the model
model_path = h2o.save_model(model=model, path='/home/rhyme/Desktop/Project/', force=True)
print(model_path)

### (Optional) Correlation Analysis

In [ ]:
copy = df.copy()
copy.head()

In [ ]:
bin_labels = ['bin1', 'bin2', 'bin3', 'bin4']
copy[['AGE','BALANCE','DAY','DURATION']] = copy[['AGE','BALANCE','DAY','DURATION']].transform(lambda x: pd.qcut(x, q=4, labels=bin_labels))

In [ ]:
copy['CAMPAIGN'] = pd.qcut(copy.CAMPAIGN, q=4, duplicates='drop', labels=bin_labels[:3])

In [ ]:
copy['PDAYS'] = pd.qcut(copy.PDAYS, q=4, duplicates='drop', labels=bin_labels[:1])

In [ ]:
copy['PREVIOUS'] = pd.qcut(copy.PREVIOUS, q=4, duplicates='drop', labels=bin_labels[:1])

In [ ]:
copy.head()

In [ ]:
copy_oh = pd.get_dummies(copy)
copy_oh.head()

In [ ]:
copy_oh.corr()['TERM_DEPOSIT_yes'].sort_values(ascending=False)[1:]

In [ ]:
results, bin_edges = pd.qcut(df.DURATION, q=4, labels=bin_labels, retbins=True)

In [ ]:
bin_edges

In [ ]:
#Strategy: Reach out to customers whose last contact duration was >= 319 or POUTCOME = success